# For  LDM-503-EFDb  

Initialize EFDs Summit and USDF

In [26]:
from lsst_efd_client import EfdClient, resample

client = EfdClient('summit_efd')
client.output = 'dataframe'
cl=client.influx_client



#usdf_client = EfdClient('summit_efd_copy')
usdf_client = EfdClient('usdf_efd')
usdf_client.output = 'dataframe'
usdf_cl=usdf_client.influx_client


## Pick five topics


In [27]:
import random
async def selectTopics(pick):
    topics = await client.get_topics()

    selected_topics = []
    loc = "summit"

    day = '2024-01-01'

    
    # want to select topics randomly but with messages so randomize all indexes 
    randoms = random.sample(range(0,len(topics)),len(topics))
    for r in randoms:
       topic = topics[r]
       if ('Test' in topic):
           next
       result = []
       if len(selected_topics) < 5:
           query=f'''SELECT * FROM "{topic}" WHERE time >  '{day}T00:00:00.000Z' and time < '{day}T00:00:30.000Z' '''
           result = await cl.query(query)

       if len(result) > 20:
           print (f"{topic} had {len(result)} messages in first 30 min of {day}")
           selected_topics.append(topic)
           if len(selected_topics) > 4:
             break
    print(f"Random selction of five '{loc}' topics {selected_topics} with messages on {day}")
    return selected_topics

## Utility function to compare reusults

In [33]:
def cmp(topic, result, sresult):
    print (f"Compare {topic}")
    problems = 0
    if len(result) != len(sresult):
        print (f"\033[91m  {topic} does not have the same number of results {len(result)}  - summit had {len(sresult)}\033[0m")
        problems = problems + 1
    else:
        for k in sresult.keys():
            if result[k].all() != sresult[k].all():
                print (f"\033[91m {topic} {key} does not match \033[0m")
                problems = problems + 1
    

In [30]:
selected_topics = await selectTopics(5) # do this once at least

lsst.sal.MTDomeTrajectory.logevent_heartbeat had 30 messages in first 30 min of 2024-01-01
lsst.sal.ATMonochromator.logevent_heartbeat had 29 messages in first 30 min of 2024-01-01
lsst.sal.MTM1M3.logevent_heartbeat had 30 messages in first 30 min of 2024-01-01
lsst.sal.ESS.lightningStrikeStatus had 30 messages in first 30 min of 2024-01-01
lsst.sal.Watcher.logevent_heartbeat had 30 messages in first 30 min of 2024-01-01
Random selction of five 'summit' topics ['lsst.sal.MTDomeTrajectory.logevent_heartbeat', 'lsst.sal.ATMonochromator.logevent_heartbeat', 'lsst.sal.MTM1M3.logevent_heartbeat', 'lsst.sal.ESS.lightningStrikeStatus', 'lsst.sal.Watcher.logevent_heartbeat'] with messages on 2024-01-01


## Get dataframes and compare summit and USDF
Doing this day by day 

In [34]:
# test gives large window 6 days

# May topics                 

data = {}
for d in range(1,7):
    if d < 10:
       day = f"2024-01-0{d}"
    else:
       day = f"2024-01-{d}"
        
    day2 = day
    print(f" Checking {day}")
    for topic in selected_topics:
        query=f'''SELECT * FROM "{topic}" WHERE time >  '{day}T00:00:00.000Z' and time < '{day2}T23:59:59.000Z' limit 1300000  '''
        # could GROUP BY time(1h) maybe ??
        result = await cl.query(query)
        usdf_result = await usdf_cl.query(query)
        cmp(topic, usdf_result, result)
        data[topic] = usdf_result
    

 Checking 2024-01-01
Compare lsst.sal.MTDomeTrajectory.logevent_heartbeat
Compare lsst.sal.ATMonochromator.logevent_heartbeat
Compare lsst.sal.MTM1M3.logevent_heartbeat
Compare lsst.sal.ESS.lightningStrikeStatus
Compare lsst.sal.Watcher.logevent_heartbeat
 Checking 2024-01-02
Compare lsst.sal.MTDomeTrajectory.logevent_heartbeat
Compare lsst.sal.ATMonochromator.logevent_heartbeat
Compare lsst.sal.MTM1M3.logevent_heartbeat
Compare lsst.sal.ESS.lightningStrikeStatus
Compare lsst.sal.Watcher.logevent_heartbeat
 Checking 2024-01-03
Compare lsst.sal.MTDomeTrajectory.logevent_heartbeat
Compare lsst.sal.ATMonochromator.logevent_heartbeat
Compare lsst.sal.MTM1M3.logevent_heartbeat
Compare lsst.sal.ESS.lightningStrikeStatus
Compare lsst.sal.Watcher.logevent_heartbeat
 Checking 2024-01-04
Compare lsst.sal.MTDomeTrajectory.logevent_heartbeat
Compare lsst.sal.ATMonochromator.logevent_heartbeat
Compare lsst.sal.MTM1M3.logevent_heartbeat
Compare lsst.sal.ESS.lightningStrikeStatus
Compare lsst.sal.Wat

In [32]:
## Reliability 

In [35]:

for topic in selected_topics:
    seqnum = data[topic]['private_seqNum']
    count = 0
    i = 1 # see if the sequence increases mostly
    prev = seqnum[0]
    while i < len(seqnum):
        if (seqnum[i] < prev):
            #print (f"Reset at {i} seqnum : {prev}, {seqnum[i]}")
            count = count + 1
        i = i + 1
    percent = 100 * ((len(seqnum) - count) / len(seqnum))
    print(f"{topic} private_seqNum increases {percent}% of the sequence")
    
    

/tmp/ipykernel_713/336143790.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prev = seqnum[0]
/tmp/ipykernel_713/336143790.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (seqnum[i] < prev):


lsst.sal.MTDomeTrajectory.logevent_heartbeat private_seqNum increases 100.0% of the sequence
lsst.sal.ATMonochromator.logevent_heartbeat private_seqNum increases 100.0% of the sequence
lsst.sal.MTM1M3.logevent_heartbeat private_seqNum increases 100.0% of the sequence
lsst.sal.ESS.lightningStrikeStatus private_seqNum increases 100.0% of the sequence
lsst.sal.Watcher.logevent_heartbeat private_seqNum increases 100.0% of the sequence
